In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Read the excel files exported:
df_ids7 = pd.read_excel('G:\IDS7 RRH XA.xlsx')
df_dt = pd.read_excel('G:\DoseTrack RRH XA reduced.xlsx')


In [ ]:
# put data from each lab into a separate dataframe:
df_ids7_RRH_XA1 = df_ids7[df_ids7['Rom/modalitet (RIS)'] == 'RRH_XA1']

# Make a list of all examination types in the dosetrack data.



# 1. Iterate through the Accession No. In the IDS7 data.
# 2. Check if there are more than one accession No. for the current patient at the current date and time.

# 3. Check if there are corresponding entries of accession No. in the dosetrack data. (DoseTrack should only have one Accession No).
# 4. Is a corresponding accession number os found in DoseTrack, use this accession No.
# 5. If there is no corresponding accession No. in DoseTrack, mark the case as having missing DoseTrack data.
# 6. Do a verification check, where all the DoseTrackdata with a valid (NRRH... etc.) accession No. exists in the IDS7 data.

# 7. Filter all the lines with data that are corrupted. 

# 8. Do a search in the DoseTrack data for duplicate dose Data. If there are lines with identical dose, remove the duplicates. There should be a minor dose difference in all CBCT acquisitions for instance.
# 9. Go through each accession No. in DoseTrack and Merge Plane A and B doses, and merge any additional CBCT examinations.

In [11]:
# Go through the IDS7 data by accession No.
# Check if there are any other accession No. on the same patient with the same booking time.

def compare_accession(df_ids7, df_dt):
    acc_no = df_ids7['Henvisinings.ID'].drop_duplicates()

    status_acc_no = [] 

    for i in acc_no:



SyntaxError: incomplete input (3663481921.py, line 10)

In [10]:
test = df_ids7['Henvisnings-ID'].drop_duplicates()
status_acc_no = [] 